In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0606,  0.0819,  0.0713, -0.0689,  0.1519,  0.0925, -0.0212, -0.0445,
         -0.1169, -0.0730],
        [ 0.0045,  0.1838, -0.0061, -0.1407,  0.1379,  0.0093,  0.0389,  0.0176,
         -0.1292, -0.0508]], grad_fn=<AddmmBackward>)

In [10]:
### 自定义块

class MLP(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [11]:
net = MLP()
net(X)

tensor([[ 0.0238,  0.3094, -0.0834, -0.0885, -0.2392, -0.0905, -0.1712, -0.1619,
         -0.1449,  0.0677],
        [ 0.1578,  0.2155,  0.0352, -0.0887, -0.2567,  0.0205, -0.2208, -0.1190,
         -0.1515, -0.0282]], grad_fn=<AddmmBackward>)

In [17]:
### 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [18]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1656, -0.0062,  0.0679,  0.1449, -0.1337,  0.1343, -0.2186, -0.0525,
         -0.1148,  0.0170],
        [ 0.1390,  0.0202,  0.1321,  0.0512, -0.2060,  0.2108, -0.1981,  0.1915,
         -0.1268,  0.0391]], grad_fn=<AddmmBackward>)

In [20]:
### forward 任意操作
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.0966, grad_fn=<SumBackward0>)

In [21]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.4182, grad_fn=<SumBackward0>)